In [25]:
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
import glob

# Decay Function
def decay(x,a,c):
    # c = tau
    return a + max_v*np.exp(-x/c)

data_path = 'ringdown raw data/11-05-2021 633/20211105-0002/'
data_files = glob.glob(data_path + '*.csv')
ring_downs = []
raw_ring_downs = []

for i in data_files:
# Pandas DataFrame of Picoscope Data
    data = pd.read_csv(i, skiprows=1) 
# Start and End position based on max Voltage and general position of user-defined end marker    
    start_full = (data['(V)'] == max(data['(V)'])).idxmax() #index of first instance of V = max(V) = True
    end_full = (data['(us)'] >= 350).idxmax() #index of first instance of time >= 350us = True
    offset = [data['(us)'][start_full], 0] #offset = [time offset, voltage offset], Voltage offset = 0
    data = data.sub(offset) #Subtract offset from dataframe; [time - time offset, Voltage - 0]
    
# Finding Range of Voltage values to determine the 90-10 value range      
    vrange = max(data['(V)'][start_full:end_full]) - min(data['(V)'][start_full:end_full])
    vmax = max(data['(V)'][start_full:end_full]) - (0.1 * vrange)
    vmin = min(data['(V)'][start_full:end_full]) + (0.1 * vrange)

# Start and End positions based on 90-10 value range
    start = (data['(V)'][start_full:end_full] <= vmax).idxmax()
    end = (data['(V)'][start_full:end_full] <= vmin).idxmax()
    max_v = max(data['(V)'][start_full:end_full])
    
# Curve Fitting
    time = data['(us)'][start_full:end_full]
    voltage = data['(V)'][start_full:end_full]
    popt, pcov = curve_fit(decay, time, voltage) #popt = optimal values for parameters, pcov = covariance of popt; scipy named variables
    
# R-Squared    
    residuals = voltage - decay(time, *popt)
    res_ss = np.sum(residuals**2)
    ss_tot = np.sum((voltage - np.mean(voltage))**2)
    r_squared = 1 - (res_ss / ss_tot)
    
# Calculating Raw ringdown time similar to software calculation    
    raw_ringdown = max(data['(us)'][start:end])-min(data['(us)'][start:end])
    
# Constant to convert tau to 90-10 ringdown time where t = tau * ln(90/10)
    tconst = np.log(9)  
    
# Compiling ring down data     
    limit = 0.99
    if r_squared >= limit:
        ring_downs.append(popt[1]*tconst)
        raw_ring_downs.append(raw_ringdown)
        #print(popt[1]*tconst, '(us), rsquared = ', r_squared)
    else:
        print('Rejecting poor fit.')
        
# Summary Statistics
print('Minimum R-Squared: ', limit)    
print('Successful ringdown fits: ', len(ring_downs),)
print('Average calculated ringdown time: ', np.mean(ring_downs), ' (us)')
print('Average raw ringdown time: ', np.mean(raw_ring_downs), ' (us)')
print('Calculated ringdown Standard Deviation: ', np.std(ring_downs), ' (us)')
print('Raw ringdown Standard Deviation: ', np.std(raw_ring_downs), ' (us)')
#print('raw min: ',min(raw_ring_downs),'raw max: ',max(raw_ring_downs))


# Picoscope returns slightly different values than currently calculated raw ringdown.
# User-defined markers determine start V, which may skew max V.
# How does Picoscope software determine end of decay?



Minimum R-Squared:  0.99
Successful ringdown fits:  32
Average calculated ringdown time:  55.993560601469355  (us)
Average raw ringdown time:  55.548248428125  (us)
Calculated ringdown Standard Deviation:  2.9438911811871926  (us)
Raw ringdown Standard Deviation:  3.404347960070465  (us)
